In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('ml_earnings.csv')

In [ ]:
df

,age,took_a_course,earnings
0,19,False,110579.0
1,28,False,142577.0
2,22,True,130520.0
3,25,True,142687.0
4,24,False,127832.0
...,...,...,...
195,35,True,180059.0
196,19,False,113031.0
197,38,False,184837.0
198,19,False,111382.0


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_percentage_error


In [ ]:
df['took_a_course'] = df['took_a_course'].astype(int)

In [ ]:
df

,age,took_a_course,earnings
0,19,0,110579.0
1,28,0,142577.0
2,22,1,130520.0
3,25,1,142687.0
4,24,0,127832.0
...,...,...,...
195,35,1,180059.0
196,19,0,113031.0
197,38,0,184837.0
198,19,0,111382.0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Define features and target for training
X_train = train_df[['age']]
y_train = train_df['took_a_course']
y_train_outcome = train_df['earnings']
X_test = test_df[['age']]
y_test_outcome = test_df['earnings']

In [ ]:
# Estimate propensity scores
ps_model = LogisticRegression().fit(X_train, y_train)
train_df['propensity_score'] = ps_model.predict_proba(X_train)[:, 1]


In [45]:
train_df['propensity_score']

,propensity_score
124,0.379272
16,0.384320
148,0.364289
93,0.379272
65,0.389392
...,...
106,0.389392
14,0.359351
92,0.379272
179,0.374251


In [ ]:
# Split into treated and control groups
treated = train_df[train_df['took_a_course'] == 1]
control = train_df[train_df['took_a_course'] == 0]

# Perform nearest neighbor matching
nn = NearestNeighbors(n_neighbors=1).fit(control[['propensity_score']])
distances, indices = nn.kneighbors(treated[['propensity_score']])
control_matches = control.iloc[indices.flatten()]

# Combine matched samples into matched_df
matched_df = pd.concat([treated, control_matches])


In [51]:
control_matches

,age,took_a_course,earnings,propensity_score
93,23,0,125074.0,0.379272
28,22,0,118836.0,0.384320
27,26,0,134860.0,0.364289
194,27,0,141665.0,0.359351
28,22,0,118836.0,0.384320
...,...,...,...,...
28,22,0,118836.0,0.384320
93,23,0,125074.0,0.379272
137,33,0,163608.0,0.330373
156,20,0,111554.0,0.394489


In [50]:
df.head(20)

,age,took_a_course,earnings
0,19,0,110579.0
1,28,0,142577.0
2,22,1,130520.0
3,25,1,142687.0
4,24,0,127832.0
5,23,0,125557.0
6,20,0,113922.0
7,23,0,124084.0
8,22,1,131905.0
9,28,0,138213.0


In [49]:
matched_df

,age,took_a_course,earnings,propensity_score
124,23,1,132636.0,0.379272
16,22,1,131140.0,0.384320
148,26,1,145022.0,0.364289
60,27,1,148107.0,0.359351
18,22,1,131868.0,0.384320
...,...,...,...,...
28,22,0,118836.0,0.384320
93,23,0,125074.0,0.379272
137,33,0,163608.0,0.330373
156,20,0,111554.0,0.394489


In [ ]:
# Prepare features and target
X_matched = matched_df[['took_a_course', 'age']]
y_matched = matched_df['earnings']

# Train regression model
reg_model = LinearRegression().fit(X_matched, y_matched)

# Display regression results
print("Regression Coefficients:", reg_model.coef_)
print("Intercept:", reg_model.intercept_)
print("Effect of taking a course on earnings:", reg_model.coef_[0])


Regression Coefficients: [10886.52540186  3932.92443942]
Intercept: 33446.256610146505
Effect of taking a course on earnings: 10886.525401863853


In [ ]:
# Predict earnings on test data
X_test_reg = test_df[['took_a_course', 'age']]
y_pred = reg_model.predict(X_test_reg)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test_outcome, y_pred)*100
print("MAPE on test data:", mape)


MAPE on test data: 1.5421315561669948


In [46]:
X=df[['took_a_course','age']]
y=df['earnings']

In [47]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [48]:
reg_model = LinearRegression().fit(X_train, y_train)

# Display regression results
print("Regression Coefficients:", reg_model.coef_)
print("Intercept:", reg_model.intercept_)
print("Effect of taking a course on earnings:", reg_model.coef_[0])  # Coefficient for 'took_a_course'
print("Effect of age on earnings:", reg_model.coef_[1])

Regression Coefficients: [10561.44058623  3951.63898731]
Intercept: 33280.44443892992
Effect of taking a course on earnings: 10561.440586232333
Effect of age on earnings: 3951.6389873105295


In [ ]:
# Predict earnings on test data
y_pred = reg_model.predict(X_test)

# Calculate MAPE on test data
mape = mean_absolute_percentage_error(y_test, y_pred)*100
print("MAPE on test data:", mape)


MAPE on test data: 1.5770167572162122
